In [2]:
import multiprocessing as mp
mp.set_start_method('spawn', force=True)
import cupy
import spacy
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm
from spacy.tokens import DocBin
from thinc.api import set_gpu_allocator, require_gpu
from multiprocessing import Process, freeze_support, set_start_method
from joblib import Parallel, delayed
from itertools import cycle

In [ ]:
rank = 0

In [4]:
def chunker(iterable, total_length, chunksize):
    return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))

In [5]:
def flatten(list_of_lists):
    "Flatten a list of lists to a combined list"
    return [item for sublist in list_of_lists for item in sublist]

In [6]:
def process_chunk(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append(lemmatize_pipe(doc))
    return preproc_pipe

In [11]:
def preprocess_parallel(texts, chunksize):
    executor = Parallel(n_jobs=7, backend='multiprocessing', prefer="processes")
    do = delayed(process_chunk)
    tasks = (do(chunk) for chunk in chunker(texts, len(texts), chunksize=chunksize))
    result = executor(tasks)
    return flatten(result)

In [8]:
    
if __name__ == '__main__':
    freeze_support()
    spacy.require_gpu()
    nlp = spacy.load("./output/model-best")

    # for ent in doc.ents:
    #     print (ent.text, ent.label_)

    listing_titles = pd.read_csv('./dataset/Listing_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')
    listing_titles = [(listing_titles["Title"][i], i) for i in range(0, len(listing_titles["Title"]))]
    
    output = open("output.txt", "w", encoding="utf-8")  # write mode

In [ ]:
    buffer = preprocess_parallel(texts = ["His friend Nicolas J. Smith is here with Bart Simpon and Fred."]*100, chunksize=5000)

In [ ]:
    print(preprocess_parallel(texts = ["His friend Nicolas J. Smith is here with Bart Simpon and Fred."]*10, chunksize=5))